## System Description
1. We have a set of COFs from a database. Each COF is characterized by a feature vector $$x_{COF} \in X \subset R^d$$ were d=14.


2. We have **two different types** of simulations to calculate **the same material property $S_{Xe/Kr}$**. Therefore, we have a Single-Task/Objective 
$$argmax_{x_{COF} \in X}[S_{Xe/Kr}(x_{COF})]$$

3. Multi-Fidelity problem. 
    1. low-fidelity  => Henry coefficient calculation - MC integration: $S_{Xe/Kr} = \frac{H_{Xe}}{H_{Kr}}$
    2. high-fidelity => GCMC mixture simulation - 80:20 (Kr:Xe) at 298 K and 1.0 bar: $S_{Xe/Kr} = \frac{n_{Xe} / n_{Kr}}{y_{Xe}/y_{Kr}}$


3. We will initialize the system with a few COFs at **both** fidelities in order to initialize the Covariance Matrix.
    1. The fist COF will be the one closest to the center of the normalized feature space
    2. The rest will be chosen to maximize diversity of the training set


4. Each surrogate model will **only train on data acquired at its level of fidelity** (Heterotopic data). $$X_{lf} \neq X_{hf} \subset X$$
    1. We  use the augmented-EI (aEI) acquisition function from [here](https://link.springer.com/content/pdf/10.1007/s00158-005-0587-0.pdf)
    2. Botorch GP surrogate model: [SingleTaskMultiFidelityGP](https://botorch.org/api/models.html#module-botorch.models.gp_regression_fidelity)
    3. Needed to use [this](https://botorch.org/api/optim.html#module-botorch.optim.fit) optimizer to correct matrix jitter
    4. Helpful [tutorial](https://botorch.org/tutorials/discrete_multi_fidelity_bo) for a similar BoTorch Model used

In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskMultiFidelityGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py # for .jld2 files
import os

# config plot settings
plt.rcParams["font.size"] = 16

In [2]:
###
#  Load Data
###
file = h5py.File("targets_and_normalized_features.jld2", "r")
# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data
y = [torch.from_numpy(np.transpose(file["henry_y"][:])), 
     torch.from_numpy(np.transpose(file["gcmc_y"][:]))]
# associated simulation costs
cost = [np.transpose(file["henry_total_elapsed_time"][:]), 
        np.transpose(file["gcmc_elapsed_time"][:])]

# total number of COFs in data set
nb_COFs = X.shape[0] 

print("raw data - \n\tX:", X.shape)
for f in range(2):
    print("\tfidelity:", f)
    print("\t\ty:", y[f].shape)
    print("\t\tcost: ", cost[f].shape)
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
	fidelity: 0
		y: torch.Size([608])
		cost:  (608,)
	fidelity: 1
		y: torch.Size([608])
		cost:  (608,)

Ensure features are normalized - 
max:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
min:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)


In [3]:
print("total high-fidelity cost:", sum(cost[1]).item(), "[min]")
print("total low-fidelity cost: ", sum(cost[0]).item(), "[min]\n")

print("average high-fidelity cost:", np.mean(cost[1]), "[min]")
print("average low-fidelity cost: ", np.mean(cost[0]), "[min]")
print("average cost ratio:\t   ", np.mean(cost[1] / cost[0]))

total high-fidelity cost: 139887.66223703226 [min]
total low-fidelity cost:  10076.305239888028 [min]

average high-fidelity cost: 230.0783918372241 [min]
average low-fidelity cost:  16.57287046034216 [min]
average cost ratio:	    13.444745568580501


## Helper Functions

#### Construct Initial Inputs

In [4]:
# find COF closest to the center of feature space
def get_initializing_COF(X):
    # center of feature space
    feature_center = np.ones(X.shape[1]) * 0.5
    # max possible distance between normalized features
    return np.argmin(np.linalg.norm(X - feature_center, axis=1))

# yields np.array([25, 494, 523])
def diverse_set(X, train_size):
    # initialize with one random point; pick others in a max diverse fashion
    ids_train = [get_initializing_COF(X)]
    # select remaining training points
    for j in range(train_size - 1):
        # for each point in data set, compute its min dist to training set
        dist_to_train_set = np.linalg.norm(X - X[ids_train, None, :], axis=2)
        assert np.shape(dist_to_train_set) == (len(ids_train), nb_COFs)
        min_dist_to_a_training_pt = np.min(dist_to_train_set, axis=0)
        assert np.size(min_dist_to_a_training_pt) == nb_COFs
        
        # acquire point with max(min distance to train set) i.e. Furthest from train set
        ids_train.append(np.argmax(min_dist_to_a_training_pt))
    assert np.size(np.unique(ids_train)) == train_size # must be unique
    return np.array(ids_train)

In [5]:
def initialize_acquired_set(X, y, nb_COFs_initialization, discrete_fidelities):
    cof_ids = diverse_set(X, nb_COFs_initialization) # np.array(ids_train)
    return torch.tensor([[f_id, cof_id] for cof_id in cof_ids for f_id in discrete_fidelities])

In [6]:
# construct feature matrix of acquired points
def build_X_train(acquired_set):
    cof_ids = [a[1] for a in acquired_set]
    f_ids = torch.tensor([a[0] for a in acquired_set])
    return torch.cat((X[cof_ids, :], f_ids.unsqueeze(dim=-1)), dim=1)

# construct output vector for acquired points
def build_y_train(acquired_set):
    return torch.tensor([y[f_id][cof_id] for f_id, cof_id in acquired_set]).unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(acquired_set):
    return torch.tensor([cost[f_id][cof_id] for f_id, cof_id in acquired_set]).unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost_fidelity(acquired_set, fidelity):
    return torch.tensor([cost[f_id][cof_id] for f_id, cof_id in acquired_set if f_id == fidelity]).unsqueeze(-1)

#### test

In [7]:
def test_initializing_functions(X, y):
    ###
    #  Construct training sets
    ###
    # list of (cof_id, fid_id)'s
    acquired_set = [[1, 10], [0, 3], [0, 4]]
    
    # Training Sets
    X_train = build_X_train(acquired_set)
    y_train = build_y_train(acquired_set)
    
    ###
    #  Test that the constructor functions are working properly
    ###
    assert np.allclose(X[10, :], X_train[0, :14])
    assert X_train[0, 14] == 1
    assert X_train[1, 14] == 0
    assert y_train[0] == y[1][10] # y[fid_id][cof_id]
    assert y_train[2] == y[0][4]
    return

test_initializing_functions(X, y)

### Surrogate Model

In [8]:
def train_surrogate_model(X_train, y_train):
    model = SingleTaskMultiFidelityGP(
        X_train, 
        y_train, 
        outcome_transform=Standardize(m=1), # m is the output dimension
        data_fidelity=X_train.shape[1] - 1
    )   
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

### Acquisition Function

In [9]:
# calculate posterior mean and variance for a given fidelity
def mu_sigma(model, X, fidelity):
    f = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) * fidelity
    X_f = torch.cat((X, f), dim=1) # last col is associated fidelity
    f_posterior = model.posterior(X_f)
    return f_posterior.mean.squeeze().detach().numpy(), np.sqrt(f_posterior.variance.squeeze().detach().numpy())

# get the current best y-value of desired_fidelity in the acquired set
def get_y_max(acquired_set, desired_fidelity):
    return np.max([y[f_id][cof_id] for f_id, cof_id in acquired_set if f_id == desired_fidelity])

In [10]:
###
# efficient multi-fidelity correlation function
# corr(y at given fidelity, y at high-fidelity)
# (see notes)
###
def mfbo_correlation_function(model, X, fidelity):
    # given fidelity
    f   = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) * fidelity
    X_f = torch.cat((X, f), dim=1) # last col is associated fidelity
    
    #  high-fidelity
    hf   = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) 
    X_hf = torch.cat((X, hf), dim=1) # last col is associated fidelity

    # combine into a single tensor
    X_all_fid = torch.cat((X_f, X_hf), dim=0)
    
    # get variance for each fidelity
    var_f = torch.flatten(model.posterior(X_f).variance)
    var_hf = torch.flatten(model.posterior(X_hf).variance) # variance
    
    # posterior covariance 
    cov = torch.diag(model(X_all_fid).covariance_matrix[:X_f.size()[0], X_f.size()[0]:])
    
    corr = cov / (torch.sqrt(var_f) * torch.sqrt(var_hf))
    return corr

In [11]:
###
#  Cost ratio
###
def estimate_cost_ratio(fidelity, acquired_set):
    avg_cost_f  = torch.mean(build_cost_fidelity(acquired_set, fidelity))
    avg_cost_hf = torch.mean(build_cost_fidelity(acquired_set, 1))
    cr = avg_cost_hf / avg_cost_f
    return cr.item()

###
#  Expected Imrovement function, only uses hf
###
def EI_hf(model, X, acquired_set):
    hf_mu, hf_sigma = mu_sigma(model, X, 1)
    y_max = get_y_max(acquired_set, 1)
    
    z = (hf_mu - y_max) / hf_sigma
    explore_term = hf_sigma * norm.pdf(z) 
    exploit_term = (hf_mu - y_max) * norm.cdf(z) 
    ei = explore_term + exploit_term
    return np.maximum(ei, np.zeros(nb_COFs))

###
#  Acquisition function
###
def acquisition_scores(model, X, fidelity, acquired_set):
    # expected improvement for high-fidelity
    ei = EI_hf(model, X, acquired_set) 
    
    # augmenting functions
    corr_f1_f0 = mfbo_correlation_function(model, X, fidelity)
    
    cr = estimate_cost_ratio(fidelity, acquired_set)

    scores = torch.from_numpy(ei) * corr_f1_f0 * cr
    return scores.detach().numpy()

def in_acquired_set(f_id, cof_id, acquired_set):
    for this_f_id, this_cof_id in acquired_set:
        if this_cof_id == cof_id and this_f_id == f_id:
            return True
    return False

# Run MFBO

In [12]:
###
#  construct initial inputs
###
discrete_fidelities = [0, 1] # set of discrete fidelities to select from
nb_COFs_initialization = 3   # at each fidelity, number of COFs to initialize with
nb_iterations = 150          # BO budget, includes initializing COFs

acquired_set = initialize_acquired_set(X, y, nb_COFs_initialization, discrete_fidelities)

X_train = build_X_train(acquired_set)
y_train = build_y_train(acquired_set)

print("Initialization - \n")
print("\tid acquired = ", [acq_[0].item() for acq_ in acquired_set])
print("\tfidelity acquired = ", [acq_[1].item() for acq_ in acquired_set])
print("\tcosts acquired = ", build_cost(acquired_set), " [min]")

print("\n\tTraining data:\n")
print("\t\t X train shape = ", X_train.shape)
print("\t\t y train shape = ", y_train.shape)
print("\t\t training feature vector = \n", X_train)

Initialization - 

	id acquired =  [0, 1, 0, 1, 0, 1]
	fidelity acquired =  [25, 25, 9, 9, 494, 494]
	costs acquired =  tensor([[ 33.2507],
        [399.7577],
        [ 28.5599],
        [344.4827],
        [ 33.2389],
        [171.9985]], dtype=torch.float64)  [min]

	Training data:

		 X train shape =  torch.Size([6, 15])
		 y train shape =  torch.Size([6, 1])
		 training feature vector = 
 tensor([[0.1500, 0.4533, 0.1088, 0.5523, 0.2651, 0.3855, 0.6004, 0.2892, 0.0000,
         0.4337, 0.1687, 0.0000, 0.0000, 0.5060, 0.0000],
        [0.1500, 0.4533, 0.1088, 0.5523, 0.2651, 0.3855, 0.6004, 0.2892, 0.0000,
         0.4337, 0.1687, 0.0000, 0.0000, 0.5060, 1.0000],
        [0.1235, 0.6225, 0.8251, 0.2767, 0.2435, 0.3542, 0.2044, 0.8856, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1235, 0.6225, 0.8251, 0.2767, 0.2435, 0.3542, 0.2044, 0.8856, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.8347, 0.9979, 0.9053, 0.0040, 0.0000

In [13]:
###
#  Run Search
###
for i in range(nb_COFs_initialization * len(discrete_fidelities), nb_iterations): 
    print("BO iteration: ", i)
    ###
    #  Train Model
    ###
    model = train_surrogate_model(X_train, y_train)

    ###
    # Acquire new (COF, fidelity) not yet acquired.
    ###
    # entry (fid_id, cof_id) is the acquisition value for fidelity f_id and cof cof_id
    the_acquisition_scores = np.array([acquisition_scores(model, X, f_id, acquired_set) for f_id in discrete_fidelities])
    # overwrite acquired COFs/fidelities with negative infinity to not choose these.
    for f_id, cof_id in acquired_set:
        the_acquisition_scores[f_id, cof_id] = - np.inf
    # select COF/fidelity with highest aquisition score.
    f_id, cof_id = np.unravel_index(np.argmax(the_acquisition_scores), np.shape(the_acquisition_scores))
    assert not in_acquired_set(f_id, cof_id, acquired_set)
    # Update acquired_set
    acq = torch.tensor([[f_id, cof_id]], dtype=int)
    acquired_set = torch.cat((acquired_set, acq))
    
    ###
    #  print useful info
    ###
    print("\tacquired COF ", cof_id, " at fidelity, ", f_id)
    print("\t\ty = ", y[f_id][cof_id].item())
    print("\t\tcost = ", cost[f_id][cof_id])
            
    # Update training sets (perform experiment and incur cost)
    X_train = build_X_train(acquired_set)
    y_train = build_y_train(acquired_set)

BO iteration:  6
Iter 10/100: 8.653264228250473
Iter 20/100: 6.2644528465905935
Iter 30/100: 4.9919128361108065
Iter 40/100: 4.179663274854121
Iter 50/100: 3.958574487794639
Iter 60/100: 3.847759198903493
Iter 70/100: 3.8023255282409116
Iter 80/100: 3.7803454009723034
Iter 90/100: 3.7697888440593252
Iter 100/100: 3.7617965839857885
	acquired COF  6  at fidelity,  0
		y =  2.572449068267546
		cost =  14.90310145219167
BO iteration:  7
Iter 10/100: 7.658788744590408
Iter 20/100: 5.597569782397579
Iter 30/100: 4.491796983302591
Iter 40/100: 3.7872467860287293
Iter 50/100: 3.5996041654689983
Iter 60/100: 3.5081607338682184
Iter 70/100: 3.468558563691321
Iter 80/100: 3.449807254609738
Iter 90/100: 3.4400909933787958
Iter 100/100: 3.4326082205894215
	acquired COF  35  at fidelity,  0
		y =  3.5769376868505414
		cost =  13.561545304457347
BO iteration:  8
Iter 10/100: 6.897599436808919
Iter 20/100: 5.082837181132256
Iter 30/100: 4.0992667432476555
Iter 40/100: 3.4589075177905304
Iter 50/100: 

	acquired COF  14  at fidelity,  0
		y =  6.87856134030866
		cost =  3.751832369963328
BO iteration:  30
Iter 10/100: 2.9152536036704277
Iter 20/100: 2.366987211696453
Iter 30/100: 2.0389233498128756
Iter 40/100: 1.789596214843989
Iter 50/100: 1.7115370194538697
	acquired COF  155  at fidelity,  0
		y =  8.124910921699056
		cost =  4.178414964675904
BO iteration:  31
Iter 10/100: 2.867414997798407
Iter 20/100: 2.333919951966667
Iter 30/100: 2.0135931927705175
Iter 40/100: 1.7622878619173579
Iter 50/100: 1.6884509390083873
	acquired COF  283  at fidelity,  0
		y =  10.776872325769247
		cost =  17.87565096616745
BO iteration:  32
Iter 10/100: 2.820570164346886
Iter 20/100: 2.3002516634241346
Iter 30/100: 1.985460317935405
Iter 40/100: 1.7400310297263397
Iter 50/100: 1.6609680732883387
	acquired COF  50  at fidelity,  0
		y =  12.195136352105475
		cost =  19.52769371668498
BO iteration:  33
Iter 10/100: 2.7760645859386437
Iter 20/100: 2.2678454808158572
Iter 30/100: 1.958384930647319
Iter

Iter 90/100: 1.478008276095686
Iter 100/100: 1.477563343178505
	acquired COF  237  at fidelity,  0
		y =  17.508435556234446
		cost =  5.643308317661285
BO iteration:  55
Iter 10/100: 2.259444479305977
Iter 20/100: 1.9247437345282827
Iter 30/100: 1.7342745990659152
Iter 40/100: 1.610131835953196
Iter 50/100: 1.528538610532603
Iter 60/100: 1.5072824430453478
Iter 70/100: 1.497507384020166
Iter 80/100: 1.4934583976092244
Iter 90/100: 1.4920441218802436
Iter 100/100: 1.4917422402602525
	acquired COF  360  at fidelity,  0
		y =  2.977726980981198
		cost =  13.41849136352539
BO iteration:  56
Iter 10/100: 2.2429964603144392
Iter 20/100: 1.9115389326045968
Iter 30/100: 1.7211839810700744
Iter 40/100: 1.599545910458209
Iter 50/100: 1.519161761511949
Iter 60/100: 1.49821396911052
Iter 70/100: 1.48864225438625
Iter 80/100: 1.4846301415884893
Iter 90/100: 1.4832419961733656
Iter 100/100: 1.4829458911860556
	acquired COF  10  at fidelity,  0
		y =  6.3961792281396095
		cost =  8.697389618555704
B

Iter 40/100: 1.4837901390667732
Iter 50/100: 1.4229798505401838
Iter 60/100: 1.4049484773605279
Iter 70/100: 1.3983389998427174
Iter 80/100: 1.3952249399400198
Iter 90/100: 1.3940169284622932
Iter 100/100: 1.3937067882765055
	acquired COF  496  at fidelity,  0
		y =  3.592821638936356
		cost =  22.02968488136927
BO iteration:  75
Iter 10/100: 2.035752062956598
Iter 20/100: 1.7565157871686237
Iter 30/100: 1.5795996663916176
Iter 40/100: 1.47675085175173
Iter 50/100: 1.4173658201738195
Iter 60/100: 1.3998592272138786
Iter 70/100: 1.3933228083984504
Iter 80/100: 1.390148169929944
Iter 90/100: 1.388873096359312
Iter 100/100: 1.3882721030929535
	acquired COF  176  at fidelity,  0
		y =  1.684115988898696
		cost =  14.408740683396658
BO iteration:  76
Iter 10/100: 2.0261675180811056
Iter 20/100: 1.7479697389838385
Iter 30/100: 1.569691400321254
Iter 40/100: 1.46823213101548
Iter 50/100: 1.4093769549404562
Iter 60/100: 1.3916895825788214
Iter 70/100: 1.385385957519588
Iter 80/100: 1.382230791

Iter 90/100: 1.354510554948848
Iter 100/100: 1.3542217832618353
	acquired COF  504  at fidelity,  0
		y =  2.3677421486178543
		cost =  54.77730476856232
BO iteration:  94
Iter 10/100: 1.9193674826967448
Iter 20/100: 1.6741240842507539
Iter 30/100: 1.511197625256206
Iter 40/100: 1.4204181686468356
Iter 50/100: 1.371069588574316
Iter 60/100: 1.35876061023041
Iter 70/100: 1.3533099128319057
Iter 80/100: 1.3504394234212855
Iter 90/100: 1.349662476982851
Iter 100/100: 1.3493733691315162
	acquired COF  189  at fidelity,  0
		y =  3.2467580347114993
		cost =  9.966407251358032
BO iteration:  95
Iter 10/100: 1.9144642688280247
Iter 20/100: 1.6703487259036207
Iter 30/100: 1.5074263258159617
Iter 40/100: 1.4171443258785148
Iter 50/100: 1.3684707807896754
Iter 60/100: 1.3561726812483792
Iter 70/100: 1.3508225453448237
Iter 80/100: 1.347949857068844
Iter 90/100: 1.34718266270461
Iter 100/100: 1.3468964992376455
	acquired COF  296  at fidelity,  0
		y =  2.2473893820932367
		cost =  11.23924303054

Iter 40/100: 1.3489462167183959
Iter 50/100: 1.307730693479874
Iter 60/100: 1.2963721966404778
Iter 70/100: 1.2913701839062746
Iter 80/100: 1.288665707526955
Iter 90/100: 1.2879087801418145
Iter 100/100: 1.2876974654437938
	acquired COF  280  at fidelity,  0
		y =  4.239684428566454
		cost =  12.143498166402182
BO iteration:  114
Iter 10/100: 1.8264388743358762
Iter 20/100: 1.6010155532700516
Iter 30/100: 1.4349419052235368
Iter 40/100: 1.3470944995241771
Iter 50/100: 1.3060331522343038
Iter 60/100: 1.2947027835725755
Iter 70/100: 1.2897040231136345
Iter 80/100: 1.2870282125106236
Iter 90/100: 1.2862749820581296
Iter 100/100: 1.286064275095031
	acquired COF  202  at fidelity,  0
		y =  8.737806654975332
		cost =  19.673663651943208
BO iteration:  115
Iter 10/100: 1.821829138551241
Iter 20/100: 1.597215440923425
Iter 30/100: 1.4308758745425163
Iter 40/100: 1.3449729027450499
Iter 50/100: 1.306279519155152
Iter 60/100: 1.2950228285781122
Iter 70/100: 1.2901006610081152
Iter 80/100: 1.287

Iter 70/100: 1.2565011200334604
Iter 80/100: 1.2549372351390562
Iter 90/100: 1.2540596411738356
Iter 100/100: 1.2538238913672117
	acquired COF  375  at fidelity,  1
		y =  18.53448594783226
		cost =  1002.6888410488765
BO iteration:  133
Iter 10/100: 1.76141037183754
Iter 20/100: 1.5474663966522761
Iter 30/100: 1.3697362020524702
Iter 40/100: 1.2733991442201322
Iter 50/100: 1.2433811392849952
Iter 60/100: 1.2301307731619822
Iter 70/100: 1.225615899968333
Iter 80/100: 1.224064400622178
Iter 90/100: 1.2230845273633817
Iter 100/100: 1.2228748372563807
	acquired COF  376  at fidelity,  1
		y =  18.396015574220662
		cost =  1178.0223760167758
BO iteration:  134
Iter 10/100: 1.754431463375999
Iter 20/100: 1.5372703597571973
Iter 30/100: 1.3485942132798876
Iter 40/100: 1.2407987084180128
Iter 50/100: 1.2134283606056167
Iter 60/100: 1.1994015623394152
Iter 70/100: 1.1949549855867023
Iter 80/100: 1.1934939972772765
Iter 90/100: 1.1924766969901832
Iter 100/100: 1.1922428704097934
	acquired COF  

The stuf below doesn't check the fidelity which is causing the indexing error (I think). The top COF (cof_id=375, y=18.53, cost=1002.68) is indeed being identified on iteration 44, but at the low-fidelity. It isn't until iteration 46 that it is being acquired at the high-fidelity (which is the one that matters.

In [14]:
###
# look at unique COFs acquired
###
# cof_ids_acquired = torch.tensor([acq[1] for acq in acquired_set])
n_unique_cofs_acquired = len(np.unique([acq[1] for acq in acquired_set]))
print("total number of unique COFs acquired", n_unique_cofs_acquired)

###
#  Iterations until top COF and accumulated 
###
cof_id_with_max_selectivity = np.argmax(y[1])
BO_iter_top_cof_acquired = float("inf") # dummy 
for i, (f_id, cof_id) in enumerate(acquired_set):
    if cof_id == cof_id_with_max_selectivity and f_id == 1:
        BO_iter_top_cof_acquired = i
        print("woo, top COF acquired!")
        print("iteration we acquire top COF = ", BO_iter_top_cof_acquired) 
        break
    elif i == len(acquired_set)-1:
        print("oh no, top COF not acquired!")


top_cof_acc_cost = sum(build_cost(acquired_set)[:BO_iter_top_cof_acquired])
print("accumulated cost up to observation of top COF = ", top_cof_acc_cost.item(), " [min]")

total number of unique COFs acquired 137
woo, top COF acquired!
iteration we acquire top COF =  132
accumulated cost up to observation of top COF =  2966.6343778570476  [min]


In [15]:
###
#  accumulated cost lags by index of 1
###
build_cost(acquired_set)[0] == sum(build_cost(acquired_set)[0])

tensor([True])

In [16]:
sum(build_cost(acquired_set)[:47]) - sum(build_cost(acquired_set)[:46])

tensor([20.1923], dtype=torch.float64)

# Store Results

In [17]:
mfbo_res = dict({'acquired_set': acquired_set.detach().numpy(),
                 'cost_acquired': build_cost(acquired_set).flatten().detach().numpy(),
                 'nb_COFs_initialization': nb_COFs_initialization,
                 'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired
                })

with open('search_results/mfbo_results_with_EI.pkl', 'wb') as file:
    pickle.dump(mfbo_res, file)